In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

## defining parameterized operations that are repeated num_time_steps times

In [2]:
N_cav = 5

In [3]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [4]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
alpha = 0.1 + 0.1j
beta = 0.1 + 0.1j
Uq = qubit_unitary(alpha)
Uqc = qubit_cavity_unitary(beta)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [6]:
qubit_unitary(0.1)

Array([[0.99875027+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.99875027+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.99875027+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        -0.04997917j, 0.99875027+0.j        ,
        0.        +0.j        , 0.        +0.

In [7]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, gamma, delta):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

### defining initial (thermal) state

In [8]:
# initial state is a thermal state coupled to a qubit in the ground state?
n_average = 1
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cav))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

In [9]:
rho_cav.shape

(5, 5)

In [10]:
rho0 = tensor(rho_cav, basis(2, 0) @ basis(2, 0).conj().T)

In [11]:
from feedback_grape.fgrape import _probability_of_a_measurement_outcome_given_a_certain_state
_probability_of_a_measurement_outcome_given_a_certain_state(rho0, 1, povm_measure_operator, [0.1, -3*jnp.pi / 2])

Array(0.88501454, dtype=float64)

### defining target state

In [12]:
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 2) + fock(N_cav, 3)) / jnp.sqrt(3), basis(2))

rho_target = psi_target @ psi_target.conj().T
rho_target.shape

(10, 10)

In [13]:
from feedback_grape.utils.fidelity import fidelity
print(fidelity(U_final=rho0, C_target=rho_target, type="density"))

0.3880207144091314


In [14]:
num_time_steps = 5
num_of_iterations = 1000
learning_rate = 0.05
# avg_photon_numer = 2 When testing kitten state

### initialize random params

In [15]:
initial_params = {
    "POVM": {
        "gamma": 0.1,
        "delta": -3 * jnp.pi / 2,
    },
    "U_q": {
        "alpha": 0.1,
    },
    "U_qc": {
        "beta": 0.1,
    }
}

In [16]:
result = optimize_pulse_with_feedback(
    U_0=rho0,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    initial_params=initial_params,
    num_time_steps=num_time_steps,
    mode="nn",
    goal="fidelity",
    optimizer="adam",
    max_iter=num_of_iterations,
    convergence_threshold=1e-20,
    learning_rate=learning_rate,
    type="density",
)

In [17]:
result

FgResult(optimized_rnn_parameters={'params': {'Dense_0': {'bias': Array([-0.04864766, -0.5895216 , -0.23658964,  1.3815178 ], dtype=float32), 'kernel': Array([[ 2.3302475e-02, -8.3136708e-01, -5.9178478e-01,  2.1965246e-01],
       [-7.6287679e-02,  1.0040702e-01, -2.6289669e-01,  3.6818293e-01],
       [-2.8205773e-02, -1.0213173e-02, -2.2085173e+00, -1.9659805e-01],
       [ 4.1897926e-02, -2.7047923e-01,  5.6667799e-01, -4.0673682e-01],
       [ 5.2603137e-02,  4.3563852e-01,  9.1423565e-01,  4.3035230e-01],
       [-3.4419898e-02,  4.6128404e-01,  2.8814490e+00,  1.6869918e+00],
       [ 1.9498125e-02,  1.0452154e+00,  4.3470573e-01, -7.4966773e-03],
       [-1.1244834e-01,  1.2251339e-01, -5.9187144e-01, -2.4661463e-01],
       [ 3.7422164e-03,  2.5852570e-01, -1.3683445e+00, -3.2037761e-02],
       [-2.0691637e-02,  2.8839245e-01, -2.2229508e-01,  2.4092446e-01],
       [-6.8426713e-02,  2.3819156e-02, -9.9923432e-01,  5.1051599e-01],
       [ 6.2144190e-02, -5.4471207e-01,  4.13

In [18]:
print(result.final_purity)

None


In [19]:
print(result.final_fidelity)

0.6947481129480583


In [20]:
print(result.iterations)

1000


In [21]:
print(result.arr_of_povm_params)

[[[0.1, -4.71238898038469], [0.1], [0.1]], [[-0.0013149690418973042, -0.6410228757900025], [1.5617688431193602], [3.9556023084849183]], [[-0.006406778098802822, -2.5344709194790607], [-3.027766327958906], [2.891238288431566]], [[-0.00495802216337226, -1.9072736255714553], [-2.6591482420471984], [1.1002029782573488]], [[-0.011580603268558712, -4.422118431958465], [-1.917912925484398], [-0.10146334077449382]]]


In [22]:
print(result.arr_of_povm_params[1])

[[-0.0013149690418973042, -0.6410228757900025], [1.5617688431193602], [3.9556023084849183]]
